In [52]:
from fastai.vision.all import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callback.tracker import SaveModelCallback
import argparse

from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy
import torchvision.transforms as T
import torch
from torch.nn.parallel import DistributedDataParallel
from torchvision.transforms.functional import *

from PIL import Image
import requests

import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch import nn

import argparse
from models.utils.joiner2 import Joiner
from models.utils.new_losses import *
from models.utils.metrics import *
from models.utils.dataLoader import *
from models.utils.datasets import *
import webdataset as wds
from models.layers.layers import PatchEmbed, Mlp

In [2]:
torch.cuda.set_device("cuda:3")
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [70]:
H = 256
W= 256
bs = 5

transform = T.Compose([
T.Resize((H,W)),
T.ToTensor(),
#T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

url1 = 'https://ogden_images.s3.amazonaws.com/www.timesrepublican.com/images/2019/05/02231710/Fox-WEB-1100x739.jpg'
im = Image.open(requests.get(url1, stream=True).raw)
img = transform(im).unsqueeze(0).to(device)
print("Input shape:", img.shape)

Input shape: torch.Size([1, 3, 256, 256])


In [71]:
img_s = 256
att_p = 32
num_att_p = 2*(256/32) # = 8x8 = 64

model_p = 8
num_model_p = 2*(256/8) # = 32x32 = 1024


In [ ]:
#model_dir = Path.home()/'Luiz/saved_models'
#net = load_learner(model_dir/'StandardCifar_Rotation_epochs-30-beta-0_lr-2e-7.pkl', cpu=False)
#model = net.model

In [ ]:
#trainable = ['fc1.weight','fc.bias']
#for name, p in model.named_parameters():
#    if name not in trainable:
#        p.requires_grad = False
#    else:
#        p.requires_grad = True

In [ ]:
#for name, p in model.named_parameters():
#    print(name, p.requires_grad)

In [72]:
path = untar_data(URLs.IMAGENETTE_320)

transform = ([*aug_transforms(),Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 splitter=RandomSplitter(),
                 get_y=parent_label,
                 item_tfms=Resize(H,W),
                 batch_tfms=transform)

dloader = data.dataloaders(path,bs=bs) 

In [54]:
critic_loss = nn.CrossEntropyLoss()

In [73]:
class Basic(nn.Module):
    def __init__(self, num_classes = 10, in_chans=3, hidden_dim=768, image_h=256, image_w=256, grid_l=8):
        super().__init__()

        self.patch_embed = PatchEmbed(
        img_size=image_h, patch_size=grid_l, in_chans=in_chans, embed_dim=hidden_dim)
        num_patches = self.patch_embed.num_patches
        
        #self.backbone = nn.Conv2d(3, hidden_dim, 3)
        #self.backbone2 = nn.Conv2d(hidden_dim, hidden_dim, 3)
        #self.fc1 = nn.Linear(hidden_dim * (image_h-4) * (image_w-4), num_classes)
        
    def forward(self, inputs):
        #h = self.backbone(inputs)
        #print(h.shape)
        #h = self.backbone2(h)
        #print(h.shape)
        #x = h.flatten(1)
        #x = self.fc1(x)
        
        x = self.patch_embed(inputs)
        
        return x

In [81]:
model = Basic().cuda()

In [82]:
learner = Learner(dloader, model, loss_func=critic_loss, metrics=[_Accuracy])

In [83]:
test = model(img.cuda())
test.shape

torch.Size([1, 1024, 768])

In [84]:
#learner.lr_find()

In [85]:
#Fitting the model
#learner.fit(10, 3e-6)